In [4]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pickle
from IPython.display import display

## Test aumentado de Dickey-Fuller
+ Prueba la estacionariedad de una serie temporal
+ Toma la primer diferencia de un modelo 
+ $H_{0}$ : si es aceptada sugiere que la serie temporal tiene una raíz unitaria por lo que es no estacionaria. Posee alguna estructura dependiente del tiempo.
+ $H_{1}$ : la hipótesis nula es rechazada, sugiere que la series es estacionaria.
+ Si el valor de la estadística del test es menor que el valor crítico más negativo entonces la hipotesis nula es rechazada
+ si p-value $> 0.05$ la hipótesis nula es aceptada

### Ejemplo

In [2]:
stock = pd.read_csv('/home/matimacazaga/Documents/Universidad/Tesis/CSV/BHIP.csv',index_col= 0, parse_dates = True)

In [7]:
cierres = list(stock['cierre'].values)
np.shape(cierres)

(3108,)

In [ ]:
result = adfuller(cierres)

print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

Como ADF Statistic es mayor que todos los valores críticos y el valor de p es mayor a 0.05 la hipotesis nula es aceptada, es decir que la serie es no estacionaria

# Código para todas las acciones

In [5]:
ticker = pickle.load(open('/home/matimacazaga/Documents/Universidad/Tesis/tickers.p','rb'))

In [6]:
df = pd.DataFrame(columns = ['ADF','p-value','Lags','5%','1%','10%'], index = ticker)

for t in ticker:
    stock = pd.read_csv('/home/matimacazaga/Documents/Universidad/Tesis/CSV/'+t+'.csv')
    if len(stock) > 2000:
        cierres = list(stock['cierre'].values)
        result = adfuller(cierres)
        df.loc[t,'ADF'] = result[0]
        df.loc[t,'p-value'] = result[1]
        df.loc[t, 'Lags'] = result[2]
        for key,value in result[4].items():
            df.loc[t,str(key)] = value

df.dropna(inplace=True)

display(df.head())
        
    
    

,ADF,p-value,Lags,5%,1%,10%
BHIP,2.43808,0.999027,27,-2.86248,-3.43247,-2.56727
BRIO,2.52237,0.999057,28,-2.86264,-3.43284,-2.56736
CADO,-1.6112,0.477344,8,-2.86262,-3.4328,-2.56735
CAPX,2.60324,0.999076,28,-2.86266,-3.43289,-2.56737
CARC,-2.62363,0.0882103,24,-2.86255,-3.43264,-2.56731


In [7]:
df.to_csv('ADF_test.csv',sep=',')

$H_{0} \rightarrow 1$

$H_{A} \rightarrow -1$

In [13]:
dict_H5 = {}
dict_H1 = {}
dict_H10 = {}
for t in df.index:
    if df.loc[t,'ADF'] > df.loc[t,'5%'] and df.loc[t,'p-value'] > 0.05:
        dict_H5[t] = 1
    else:
        dict_H5[t] = 0
    if df.loc[t,'ADF'] > df.loc[t,'1%'] and df.loc[t,'p-value'] > 0.05:
        dict_H1[t] = 1
    else:
        dict_H1[t] = 0
    if df.loc[t,'ADF'] > df.loc[t,'10%'] and df.loc[t,'p-value'] > 0.05:
        dict_H10[t] = 1
    else:
        dict_H10[t] = 0


In [23]:
print('Acciones no estacionarias con nivel de significancia menor a 1%:',np.sum([dict_H1[t] for t in df.index])/len(df.index)*100,'%')
print('Acciones no estacionarias con nivel de significancia menor a 10%:',np.sum([dict_H10[t] for t in df.index])/len(df.index)*100,'%')
print('Acciones no estacionarias con nivel de significancia menor a 5%:',np.sum([dict_H5[t] for t in df.index])/len(df.index)*100,'%')

Acciones no estacionarias con nivel de significancia menor a 1%: 97.9166666667 %
Acciones no estacionarias con nivel de significancia menor a 10%: 91.6666666667 %
Acciones no estacionarias con nivel de significancia menor a 5%: 97.9166666667 %


El nivel de significancia indica la probabilidad de que el resultado sea una **casualidad estadística**

In [29]:
list(dict_H1.items())[0][0]

'BHIP'

In [31]:
for i,t in enumerate(df.index):
    if dict_H1[t] == 0:
        print(t)

PATA


In [32]:
for i,t in enumerate(df.index):
    if dict_H10[t] == 0:
        print(t)

CARC
PATA
POLL
APBR


In [33]:
for i,t in enumerate(df.index):
    if dict_H5[t] == 0:
        print(t)

PATA
